# **Dự đoán điểm chuẩn đại học - Predict Standard Point**

In [1]:
import numpy as np
import pandas as pd

In [2]:
def val_count(data, subject):
    result = {}
    num = data.shape[0]
    for index in range(num):
        result[data['diem'][index]] = data[subject][index]
    return result

### **1. Lấy dữ liệu năm 2021**

In [3]:
data2021_1 = pd.read_csv("./data/data_2021(01).csv")
data2021_1.head()

,diem,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd
0,0.00,26,5,10,25,4,94,24
1,0.25,10,2,1,1,7,1,0
2,0.50,45,1,1,1,24,3,1
3,0.75,68,3,20,11,125,9,2
4,1.00,23,14,26,37,380,11,2


In [4]:
point_count2021 = \
{\
'ngu_van'   : val_count(data2021_1, 'ngu_van'),
'vat_li'    : val_count(data2021_1, 'vat_li'),
'hoa_hoc'   : val_count(data2021_1, 'hoa_hoc'),
'sinh_hoc'  : val_count(data2021_1, 'sinh_hoc'),
'lich_su'   : val_count(data2021_1, 'lich_su'),
'dia_ly'    : val_count(data2021_1, 'dia_ly'),
'gdcd'      : val_count(data2021_1, 'gdcd'),
}

In [5]:
data2021_2 = pd.read_csv("./data/data_2021(02).csv")
data2021_2.head()

,diem,toan,ngoai_ngu
0,0.0,1,1
1,0.2,0,0
2,0.4,0,0
3,0.6,11,3
4,0.8,22,34


In [6]:
point_count2021['toan'] = val_count(data2021_2, 'toan')
point_count2021['ngoai_ngu'] = val_count(data2021_2, 'ngoai_ngu')

### **2. Lấy dữ liệu năm 2022**

In [7]:
data_2022 = pd.read_csv('new_data1.csv')
data_2022.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,phan_loai,tong_diem_3_mon,khoi_thi
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75,THPT_KHXH,13.75,C00
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25,THPT_KHXH,24.40,A07
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75,THPT_KHXH,20.25,C00
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50,THPT_KHXH,23.50,D01
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25,THPT_KHXH,25.00,C00


In [8]:
point_count2022 = \
{\
'toan'      : dict(data_2022['toan'].value_counts()),
'ngoai_ngu' : dict(data_2022['ngoai_ngu'].value_counts()),
'ngu_van'   : dict(data_2022['ngu_van'].value_counts()),
'vat_li'    : dict(data_2022['vat_li'].value_counts()),
'hoa_hoc'   : dict(data_2022['hoa_hoc'].value_counts()),
'sinh_hoc'  : dict(data_2022['sinh_hoc'].value_counts()),
'lich_su'   : dict(data_2022['lich_su'].value_counts()),
'dia_ly'    : dict(data_2022['dia_ly'].value_counts()),
'gdcd'      : dict(data_2022['gdcd'].value_counts()),
}

### **3. Tỷ lệ học sinh trên X năm 2022 so với 2021**

In [9]:
def rate(subject_2021, subject_2022, min_point):
    sum2021 = 0
    sum2022 = 0
    full2021 = 0
    full2022 = 0
    for point in subject_2021:
        try:
            full2021 += subject_2021[point]
            full2022 += subject_2022[point]
            if point >= min_point:
                sum2021 += subject_2021[point]
                sum2022 += subject_2022[point]
        except:
            continue

    rate = (sum2022 / full2022) / (sum2021 / full2021)
    return rate

In [10]:
def dic_rate(data1, data2, min_point):
    res = {}
    for sub in data1:
        res[sub] = rate(data1[sub], data2[sub], min_point)
    return res

In [11]:
rate_8_subject = dic_rate(point_count2021, point_count2022, 8)
rate_9_subject = dic_rate(point_count2021, point_count2022, 9)
print(rate_8_subject)
print(rate_9_subject)

{'ngu_van': 1.1379575636191872, 'vat_li': 1.2412913283893174, 'hoa_hoc': 1.119856329497466, 'sinh_hoc': 0.7034398588379197, 'lich_su': 2.5791232451566017, 'dia_ly': 0.7583814733187179, 'gdcd': 0.865617250804654, 'toan': 0.8444845424820955, 'ngoai_ngu': 0.5021423418760592}
{'ngu_van': 1.6098835005930603, 'vat_li': 3.13302756493306, 'hoa_hoc': 1.501100346494167, 'sinh_hoc': 0.330759908359931, 'lich_su': 2.609589843475174, 'dia_ly': 0.6765593325736329, 'gdcd': 0.5592383594039315, 'toan': 0.5659612792143117, 'ngoai_ngu': 0.33937490400587034}


### **4. Tính điểm chuẩn (dự đoán) qua tỉ lệ học sinh trên 8 điểm**

In [12]:
khoiA00 = ['toan','vat_li','hoa_hoc']
khoiA01 = ['toan', 'vat_li', 'ngoai_ngu']
khoiA02 = ['toan', 'vat_li', 'sinh_hoc']
khoiA07 = ['toan', 'lich_su', 'dia_ly']
khoiB00 = ['toan', 'sinh_hoc', 'hoa_hoc']
khoiB03 = ['toan', 'sinh_hoc', 'ngu_van']
khoiB08 = ['toan', 'sinh_hoc', 'ngoai_ngu']
khoiC00 = ['ngu_van', 'lich_su', 'dia_ly']
khoiC01 = ['ngu_van', 'toan', 'vat_li']
khoiC02 = ['ngu_van', 'toan', 'hoa_hoc']
khoiD01 = ['ngu_van', 'toan', 'ngoai_ngu']
khoiD07 = ['toan', 'hoa_hoc', 'ngoai_ngu'] 
khoiD08 = ['toan', 'sinh_hoc', 'ngoai_ngu']


khoi_thi = {'A00': khoiA00, 'A01': khoiA01, 'A02': khoiA02, 'A07': khoiA07, 
            'B00': khoiB00, 'B03': khoiB03, 'B08': khoiB08, 
            'C00': khoiC00, 'C01': khoiC01, 'C02': khoiC02, 
            'D01': khoiD01, 'D07': khoiD07}

In [13]:
khoi_count = data_2022['khoi_thi'].value_counts()
print(khoi_count)

C00    379394
A07    190098
D01    123663
A00     96795
C01     51762
C02     51064
A01     45924
B00     20346
D07     19563
B03      8101
A02      2863
B08      2422
Name: khoi_thi, dtype: int64


In [34]:
def predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate8, rate9):
    diem2022 = 0
    rate_count = {}
    sum_count = 0
    for khoi in khoi_xet_tuyen:
        sum_count += khoi_count[khoi]
        subjects = khoi_thi[khoi]
        rate_khoi = 0
        for sub in subjects:
            rate_khoi += rate8[sub] * 0.97 + rate9[sub] * 0.03
        rate_count[khoi] = rate_khoi / len(subjects)
    for khoi in khoi_xet_tuyen:
        diem2022 += rate_count[khoi] * khoi_count[khoi] / sum_count
    
    diem2022 = diem2022 * diem2021
    return diem2022

In [35]:
# Ngành: Nhóm ngành CNTT - trường ĐH Khoa học tự nhiên - DHQG TP.HCM --> 27.2
diem2021 = 27.4
khoi_xet_tuyen = ['A00', 'A01', 'B08', 'D07']
diem2022 = predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate_8_subject, rate_9_subject)
print(diem2022)

27.174712448444073


In [36]:
# Ngành: Công nghệ sinh học - trường ĐH Khoa học tự nhiên - DHQG TP.HCM -->23.75
diem2021 = 25.5
khoi_xet_tuyen = ['A02', 'B00']
diem2022 = predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate_8_subject, rate_9_subject)
print(diem2022)

22.782139949872334


In [37]:
# Ngành: Hóa học - trường ĐH Khoa học tự nhiên - DHQG TP.HCM -->23.75
diem2021 = 24.5
khoi_xet_tuyen = ['A00', 'B00']
diem2022 = predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate_8_subject, rate_9_subject)
print(diem2022)

25.808695545268623


In [40]:
# Ngành: Kiểm toán - Đại học Kinh tế quốc dân --> 28.15
diem2021 = 28.1
khoi_xet_tuyen = ['A00', 'A01', 'D07']
diem2022 = predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate_8_subject, rate_9_subject)
print(diem2022)

28.001717409695782


In [41]:
# Ngành: Răng-Hàm-Mặt - Đại học Y Hà Nội --> 27.7
diem2021 = 28.1
khoi_xet_tuyen = ['B00']
diem2022 = predict_score(diem2021, khoi_count, khoi_xet_tuyen, rate_8_subject, rate_9_subject)
print(diem2022)

24.91235431103725
